# ELO Model Training - Hyperparameter Grid Search

This notebook:
1. Loads the 648 hyperparameter configs from Ruby
2. Trains ELO model for each config
3. Tracks RMSE, MAE, R² for each
4. Saves results and identifies best config

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sys
import os

# Add parent directory for utils imports
sys.path.insert(0, os.path.dirname(os.getcwd()))

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# EloModel Class (self-contained for portability)
class EloModel:
    def __init__(self, params):
        self.params = params
        self.ratings = {}
        self.rating_history = []
    
    def initialize_ratings(self, teams, divisions=None):
        """Initialize team ratings based on division tier."""
        initial = self.params.get('initial_rating', 1500)
        division_ratings = {'D1': initial + 100, 'D2': initial, 'D3': initial - 100}
        
        for i, team in enumerate(teams):
            if divisions is not None and i < len(divisions):
                div = divisions.iloc[i] if hasattr(divisions, 'iloc') else divisions[i]
                self.ratings[team] = division_ratings.get(div, initial)
            else:
                self.ratings[team] = initial
    
    def calculate_expected_score(self, team_elo, opponent_elo):
        return 1 / (1 + 10 ** ((opponent_elo - team_elo) / 400))
    
    def calculate_mov_multiplier(self, goal_diff):
        mov = self.params.get('mov_multiplier', 0)
        if mov == 0:
            return 1.0
        if self.params.get('mov_method', 'logarithmic') == 'linear':
            return 1 + (abs(goal_diff) * mov)
        return 1 + (np.log(abs(goal_diff) + 1) * mov)
    
    def get_actual_score(self, outcome):
        if outcome in ['RW', 'W', 1]:
            return 1.0
        elif outcome == 'OTW':
            return self.params.get('ot_win_multiplier', 0.75)
        elif outcome == 'OTL':
            return 1 - self.params.get('ot_win_multiplier', 0.75)
        return 0.0
    
    def adjust_for_context(self, team_elo, is_home, rest_time, travel_dist, injuries):
        adjusted = team_elo
        if is_home:
            adjusted += self.params.get('home_advantage', 0)
        if rest_time <= 1:
            adjusted -= self.params.get('b2b_penalty', 0)
        if not is_home and travel_dist > 0:
            adjusted -= (travel_dist / 1000) * 15
        adjusted -= injuries * 25
        return adjusted
    
    def update_ratings(self, game):
        home_team = game['home_team']
        away_team = game['away_team']
        
        home_elo = self.ratings.get(home_team, 1500)
        away_elo = self.ratings.get(away_team, 1500)
        
        # Get context values with defaults
        home_rest = game.get('home_rest', 2)
        away_rest = game.get('away_rest', 2)
        away_travel = game.get('away_travel_dist', game.get('travel_distance', 0))
        home_injuries = game.get('home_injuries', game.get('injuries', 0))
        away_injuries = game.get('away_injuries', game.get('injuries', 0))
        
        home_adj = self.adjust_for_context(home_elo, True, home_rest, 0, home_injuries)
        away_adj = self.adjust_for_context(away_elo, False, away_rest, away_travel, away_injuries)
        
        rest_diff = home_rest - away_rest
        home_adj += rest_diff * self.params.get('rest_advantage_per_day', 0)
        
        home_expected = self.calculate_expected_score(home_adj, away_adj)
        
        # Handle different outcome column names
        if 'home_outcome' in game:
            home_actual = self.get_actual_score(game['home_outcome'])
        elif 'home_win' in game:
            home_actual = 1.0 if game['home_win'] else 0.0
        else:
            home_actual = 1.0 if game['home_goals'] > game['away_goals'] else 0.0
        
        goal_diff = game['home_goals'] - game['away_goals']
        mov_mult = self.calculate_mov_multiplier(goal_diff)
        
        k = self.params.get('k_factor', 32) * mov_mult
        self.ratings[home_team] = home_elo + k * (home_actual - home_expected)
        self.ratings[away_team] = away_elo + k * ((1 - home_actual) - (1 - home_expected))
        
        self.rating_history.append({
            'home_team': home_team, 'away_team': away_team,
            'home_rating': self.ratings[home_team],
            'away_rating': self.ratings[away_team]
        })
    
    def predict_goals(self, game):
        home_team = game['home_team']
        away_team = game['away_team']
        
        home_elo = self.ratings.get(home_team, 1500)
        away_elo = self.ratings.get(away_team, 1500)
        
        home_rest = game.get('home_rest', 2)
        away_rest = game.get('away_rest', 2)
        away_travel = game.get('away_travel_dist', game.get('travel_distance', 0))
        home_injuries = game.get('home_injuries', game.get('injuries', 0))
        away_injuries = game.get('away_injuries', game.get('injuries', 0))
        
        home_adj = self.adjust_for_context(home_elo, True, home_rest, 0, home_injuries)
        away_adj = self.adjust_for_context(away_elo, False, away_rest, away_travel, away_injuries)
        
        rest_diff = home_rest - away_rest
        home_adj += rest_diff * self.params.get('rest_advantage_per_day', 0)
        
        home_win_prob = self.calculate_expected_score(home_adj, away_adj)
        expected_diff = (home_win_prob - 0.5) * 12
        
        home_goals = 3.0 + (expected_diff / 2)
        away_goals = 3.0 - (expected_diff / 2)
        return home_goals, away_goals
    
    def fit(self, games_df):
        teams = pd.concat([games_df['home_team'], games_df['away_team']]).unique()
        if 'division' in games_df.columns:
            divisions = games_df.groupby('home_team')['division'].first()
            self.initialize_ratings(teams, divisions)
        else:
            self.initialize_ratings(teams)
        
        for _, game in games_df.iterrows():
            self.update_ratings(game)
    
    def evaluate(self, games_df):
        predictions, actuals = [], []
        for _, game in games_df.iterrows():
            home_pred, _ = self.predict_goals(game)
            predictions.append(home_pred)
            actuals.append(game['home_goals'])
        
        rmse = mean_squared_error(actuals, predictions, squared=False)
        mae = mean_absolute_error(actuals, predictions)
        r2 = r2_score(actuals, predictions) if len(set(actuals)) > 1 else 0.0
        return {'rmse': rmse, 'mae': mae, 'r2': r2}

print("EloModel class loaded successfully!")

## Load Data

In [ ]:
# Load hyperparameter grid (generated by Ruby)
# Try multiple possible paths
config_paths = [
    '../data/model3_elo_grid.csv',           # DeepNote: uploaded to python/data/
    '../../output/hyperparams/model3_elo_grid.csv',  # Local: from training/ folder
    'model3_elo_grid.csv',                    # Current directory
]

configs_df = None
for path in config_paths:
    if os.path.exists(path):
        configs_df = pd.read_csv(path)
        print(f"✅ Loaded configs from: {path}")
        break

if configs_df is None:
    print("⚠️ No config file found. Generating default grid...")
    # Generate default grid if no file found
    from itertools import product
    k_factors = [20, 32, 40]
    home_advantages = [50, 100, 150]
    mov_multipliers = [0, 1.0, 1.5]
    rest_advantages = [0, 10]
    b2b_penalties = [0, 50]
    
    configs = []
    for i, (k, h, m, r, b) in enumerate(product(k_factors, home_advantages, mov_multipliers, rest_advantages, b2b_penalties)):
        configs.append({
            'experiment_id': f'elo_{i+1:03d}',
            'k_factor': k, 'home_advantage': h, 'mov_multiplier': m,
            'mov_method': 'logarithmic', 'rest_advantage_per_day': r, 'b2b_penalty': b,
            'initial_rating': 1500, 'ot_win_multiplier': 0.75
        })
    configs_df = pd.DataFrame(configs)
    print(f"Generated {len(configs_df)} default configurations")

print(f"\n📊 Total configurations: {len(configs_df)}")
configs_df.head()

In [ ]:
# Load hockey game data
# Try multiple possible paths
data_paths = [
    '../data/hockey_data.csv',      # DeepNote: uploaded to python/data/
    '../../data/hockey_data.csv',   # Local: from training/ folder
    'hockey_data.csv',              # Current directory
]

games_df = None
for path in data_paths:
    if os.path.exists(path):
        games_df = pd.read_csv(path)
        print(f"✅ Loaded games from: {path}")
        break

if games_df is None:
    raise FileNotFoundError(
        "❌ Hockey data not found! Upload hockey_data.csv to python/data/ folder.\n"
        "Expected columns: home_team, away_team, home_goals, away_goals, game_date, division"
    )

# CRITICAL: Sort by game date (ELO requires chronological order)
date_col = None
for col in ['game_date', 'date', 'Date', 'game_datetime']:
    if col in games_df.columns:
        date_col = col
        break

if date_col:
    games_df = games_df.sort_values(date_col).reset_index(drop=True)
    print(f"✅ Sorted by: {date_col}")
else:
    print("⚠️ No date column found - assuming data is already chronological")

print(f"\n📊 Loaded {len(games_df)} games")
print(f"📋 Columns: {list(games_df.columns)}")
games_df.head()

## Time Series Split for Validation

In [ ]:
# Use 80/20 train/test split (chronological)
split_idx = int(len(games_df) * 0.8)
train_df = games_df[:split_idx]
test_df = games_df[split_idx:]

print(f"Train: {len(train_df)} games")
print(f"Test: {len(test_df)} games")

## Grid Search Loop

In [ ]:
results = []

# Loop through all configs (this will take a while - 648 iterations)
for idx, row in tqdm(configs_df.iterrows(), total=len(configs_df), desc="Training ELO models"):
    try:
        # Convert row to parameters dict
        params = row.to_dict()
        experiment_id = params.pop('experiment_id')
        
        # Initialize model
        model = EloModel(params)
        
        # Train on training set
        model.fit(train_df)
        
        # Evaluate on test set
        metrics = model.evaluate(test_df)
        
        # Store results
        results.append({
            'experiment_id': experiment_id,
            'rmse': metrics['rmse'],
            'mae': metrics['mae'],
            'r2': metrics['r2'],
            'status': 'completed',
            **params
        })
        
    except Exception as e:
        print(f"Error in experiment {experiment_id}: {e}")
        results.append({
            'experiment_id': experiment_id,
            'rmse': np.nan,
            'mae': np.nan,
            'r2': np.nan,
            'status': 'failed',
            **params
        })

# Convert to DataFrame
results_df = pd.DataFrame(results)
print(f"\nCompleted {len(results_df)} experiments")
print(f"Failed: {results_df['status'].value_counts().get('failed', 0)}")

## Save Results

In [ ]:
# Save results with metrics
output_path = '../data/model3_elo_results.csv'  # Save to python/data/ for portability
results_df.to_csv(output_path, index=False)
print(f"✅ Saved results to: {output_path}")

# Display summary
completed = results_df[results_df['status'] == 'completed']
print(f"\n📊 Summary:")
print(f"   Completed: {len(completed)}")
print(f"   Best RMSE: {completed['rmse'].min():.3f}")
print(f"   Mean RMSE: {completed['rmse'].mean():.3f}")

## Analyze Best Configurations

In [ ]:
# Find best configs by RMSE
best_configs = results_df.nsmallest(10, 'rmse')
print("\nTop 10 Configurations by RMSE:")
print(best_configs[['experiment_id', 'rmse', 'mae', 'r2', 'k_factor', 'home_advantage', 
                     'mov_multiplier', 'rest_advantage_per_day', 'b2b_penalty']])

In [ ]:
# Best overall config
best = results_df.loc[results_df['rmse'].idxmin()]
print(f"\n🏆 BEST CONFIGURATION:")
print(f"   Experiment ID: {best['experiment_id']}")
print(f"   RMSE: {best['rmse']:.3f}")
print(f"   MAE: {best['mae']:.3f}")
print(f"   R²: {best['r2']:.3f}")
print(f"\n   Parameters:")
print(f"   - k_factor: {best['k_factor']}")
print(f"   - home_advantage: {best['home_advantage']}")
print(f"   - mov_multiplier: {best['mov_multiplier']}")
print(f"   - mov_method: {best['mov_method']}")
print(f"   - rest_advantage_per_day: {best['rest_advantage_per_day']}")
print(f"   - b2b_penalty: {best['b2b_penalty']}")

## Visualize Results

In [ ]:
# Distribution of RMSE scores
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].hist(results_df['rmse'].dropna(), bins=50, edgecolor='black')
axes[0].axvline(best['rmse'], color='red', linestyle='--', linewidth=2, label=f"Best: {best['rmse']:.3f}")
axes[0].set_xlabel('RMSE')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of RMSE Scores')
axes[0].legend()

axes[1].hist(results_df['r2'].dropna(), bins=50, edgecolor='black', color='green', alpha=0.7)
axes[1].axvline(best['r2'], color='red', linestyle='--', linewidth=2, label=f"Best: {best['r2']:.3f}")
axes[1].set_xlabel('R² Score')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of R² Scores')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Parameter importance heatmap
param_cols = [col for col in ['k_factor', 'home_advantage', 'mov_multiplier', 'rest_advantage_per_day', 'b2b_penalty'] 
              if col in results_df.columns]

if param_cols:
    corr = results_df[param_cols + ['rmse']].corr()['rmse'].drop('rmse')
    
    plt.figure(figsize=(8, 5))
    corr.abs().sort_values().plot(kind='barh', color='steelblue')
    plt.xlabel('Correlation with RMSE (absolute)')
    plt.title('Hyperparameter Importance')
    plt.tight_layout()
    plt.show()
else:
    print("No parameter columns found for correlation analysis")

## Train Final Model with Best Config

In [ ]:
# Train on full dataset with best parameters
best_params = best.drop(['experiment_id', 'rmse', 'mae', 'r2', 'status']).to_dict()
final_model = EloModel(best_params)
final_model.fit(games_df)

print("Final model trained on full dataset")
print(f"Final team ratings:")
sorted_ratings = sorted(final_model.ratings.items(), key=lambda x: x[1], reverse=True)
for team, rating in sorted_ratings[:10]:
    print(f"  {team}: {rating:.1f}")

## Generate Predictions for Submission

In [ ]:
# Generate predictions for test/submission games
def generate_predictions(model, test_games_df, output_path=None):
    """Generate goal predictions for a set of games."""
    predictions = []
    for _, game in test_games_df.iterrows():
        home_pred, away_pred = model.predict_goals(game)
        predictions.append({
            'game_id': game.get('game_id', _),
            'home_team': game['home_team'],
            'away_team': game['away_team'],
            'home_goals_pred': round(home_pred, 2),
            'away_goals_pred': round(away_pred, 2),
        })
    
    predictions_df = pd.DataFrame(predictions)
    
    if output_path:
        predictions_df.to_csv(output_path, index=False)
        print(f"✅ Predictions saved to: {output_path}")
    
    return predictions_df

# Example: Generate predictions on test set
test_predictions = generate_predictions(final_model, test_df)
print(f"\n📊 Test Set Predictions ({len(test_predictions)} games):")
test_predictions.head(10)

## Save Best Model Configuration

In [ ]:
import json

# Save best configuration as JSON for reuse
best_config = {
    'model': 'ELO',
    'experiment_id': best['experiment_id'],
    'metrics': {
        'rmse': float(best['rmse']),
        'mae': float(best['mae']),
        'r2': float(best['r2'])
    },
    'params': {k: (float(v) if isinstance(v, (int, float, np.floating, np.integer)) else v) 
               for k, v in best_params.items()},
    'team_ratings': {k: round(v, 1) for k, v in final_model.ratings.items()}
}

# Save to JSON
config_path = '../data/best_elo_config.json'
with open(config_path, 'w') as f:
    json.dump(best_config, f, indent=2)

print(f"✅ Best configuration saved to: {config_path}")
print(f"\n🏆 FINAL MODEL SUMMARY:")
print(f"   RMSE: {best['rmse']:.3f}")
print(f"   MAE: {best['mae']:.3f}")  
print(f"   R²: {best['r2']:.3f}")
print(f"\n   Best Parameters:")
for key, val in best_params.items():
    print(f"   - {key}: {val}")